In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Conv1D, Dense, Dropout, MaxPooling1D, Flatten
from tensorflow.keras.optimizers import Adam, Nadam, Adagrad
# from tensorflow.keras.callbacks import EarlyStopping


In [50]:
df = pd.read_csv("sensor-nodes.csv")
df.head(10)


,Node_ID,Timestamp,IP_Address,Packet_Rate,Packet_Drop_Rate,Packet_Duplication_Rate,Data_Throughput,Signal_Strength,SNR,Battery_Level,...,Number_of_Neighbors,Route_Request_Frequency,Route_Reply_Frequency,Data_Transmission_Frequency,Data_Reception_Frequency,Error_Rate,CPU_Usage,Memory_Usage,Bandwidth,Is_Malicious
0,1,01-02-23 0:00,192.168.119.138,52.018229,2.727317,0.955687,130.967180,-52.059232,23.974265,82.982598,...,3,11.323044,11.458199,20.670539,14.385439,0.109146,26.376693,29.186341,76.811986,0
1,2,01-02-23 0:01,192.168.225.56,59.504648,1.435058,1.228460,82.634358,-57.754723,22.572052,66.734853,...,6,8.277362,8.832980,15.555122,14.316681,0.077136,34.963429,34.203177,112.495912,0
2,3,01-02-23 0:02,192.168.187.210,34.118204,1.320905,1.015245,61.649928,-52.399037,15.667076,74.593605,...,5,8.196374,8.330227,19.470748,11.792084,0.052993,48.934585,31.682662,117.526109,0
3,4,01-02-23 0:03,192.168.78.245,39.643055,1.988791,0.784562,69.704181,-53.727443,20.286962,76.710564,...,6,10.683766,6.525032,16.228955,8.895409,0.087810,47.045600,37.470027,123.058913,0
4,5,01-02-23 0:04,192.168.233.229,40.212679,1.005121,1.412193,123.445612,-49.696620,24.318538,84.477890,...,7,11.176823,10.617432,12.257423,10.168739,0.068823,33.394354,26.582970,71.696911,0
5,6,01-02-23 0:05,192.168.230.20,42.539308,1.777329,1.226797,140.306629,-48.164891,22.562783,80.792853,...,3,9.100424,10.912167,17.039896,17.098904,0.095630,41.592922,37.011924,135.173141,0
6,7,01-02-23 0:06,192.168.201.178,34.371305,2.122728,1.238309,82.759149,-46.749541,22.199859,83.896476,...,5,5.514884,9.764031,14.729525,14.537270,0.112991,51.911647,33.413317,113.722314,0
7,8,01-02-23 0:07,192.168.0.73,39.421713,1.632756,1.070672,128.744279,-47.965449,22.017101,73.018853,...,3,8.785059,7.424502,14.028956,16.584326,0.150733,43.373535,32.514071,101.301618,0
8,9,01-02-23 0:08,192.168.35.14,64.392638,1.771348,0.614431,105.554913,-45.030005,19.406187,100.505321,...,4,9.652375,9.578266,20.649000,13.220874,0.133769,41.662683,36.024818,130.214575,0
9,10,01-02-23 0:09,192.168.94.186,71.586404,2.874885,1.175556,109.901877,-43.949902,27.354364,76.642262,...,3,8.413089,14.570255,15.308871,14.962364,0.088105,39.467362,31.086755,130.360976,0


In [51]:
X = df.drop(columns=['Node_ID', 'Timestamp', 'IP_Address', 'Is_Malicious'])
y = df['Is_Malicious']


In [52]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [53]:
# LSTM
X_train_val_lstm, X_test_lstm, y_train_val_lstm, y_test_lstm = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42)

X_train_lstm, X_val_lstm, y_train_lstm, y_val_lstm = train_test_split(
    X_train_val_lstm, y_train_val_lstm, test_size=0.25, random_state=42)

print("Shape of X_train:", X_train_lstm.shape)
print("Shape of X_val:", X_val_lstm.shape)
print("Shape of X_test:", X_test_lstm.shape)
print("Shape of y_train:", y_train_lstm.shape)
print("Shape of y_val:", y_val_lstm.shape)
print("Shape of y_test:", y_test_lstm.shape)


Shape of X_train: (6000, 17)
Shape of X_val: (2000, 17)
Shape of X_test: (2000, 17)
Shape of y_train: (6000,)
Shape of y_val: (2000,)
Shape of y_test: (2000,)


In [54]:
sequence_length = X_train_lstm.shape[1]
num_features = 1

X_train_reshaped_lstm = np.expand_dims(X_train_lstm, axis=-1)
X_val_reshaped_lstm = np.expand_dims(X_val_lstm, axis=-1)
X_test_reshaped_lstm = np.expand_dims(X_test_lstm, axis=-1)

print("Sequence length: ", sequence_length)
print("Num of features: ", num_features)


Sequence length:  17
Num of features:  1


In [55]:
lstm_model = Sequential()
lstm_model.add(LSTM(units=64, input_shape=(sequence_length, num_features),
               kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001)))
lstm_model.add(Dropout(0.5))


In [56]:
# Adam Optimizer
lstm_adam_optimizer = Adam(learning_rate=0.001)

lstm_model.compile(optimizer=lstm_adam_optimizer,
                   loss='binary_crossentropy', metrics=['accuracy'])

lstm_adam_history = lstm_model.fit(X_train_reshaped_lstm, y_train_lstm,
                                   epochs=30, batch_size=32, validation_data=(X_val_reshaped_lstm, y_val_lstm))


Epoch 1/30


188/188 [==============================] - 3s 7ms/step - loss: 0.5256 - accuracy: 0.0792 - val_loss: 0.1274 - val_accuracy: 0.0000e+00
Epoch 2/30
188/188 [==============================] - 1s 5ms/step - loss: 0.4433 - accuracy: 0.2900 - val_loss: 0.0835 - val_accuracy: 0.0000e+00
Epoch 3/30
188/188 [==============================] - 1s 5ms/step - loss: 0.4071 - accuracy: 0.3362 - val_loss: 0.0723 - val_accuracy: 0.0000e+00
Epoch 4/30
188/188 [==============================] - 1s 5ms/step - loss: 0.4052 - accuracy: 0.3695 - val_loss: 0.0689 - val_accuracy: 0.0000e+00
Epoch 5/30
188/188 [==============================] - 1s 5ms/step - loss: 0.4002 - accuracy: 0.3750 - val_loss: 0.0601 - val_accuracy: 0.0000e+00
Epoch 6/30
188/188 [==============================] - 1s 8ms/step - loss: 0.3977 - accuracy: 0.3853 - val_loss: 0.0483 - val_accuracy: 0.0000e+00
Epoch 7/30
188/188 [==============================] - 1s 8ms/step - loss: 0.3952 - accuracy: 0.4153 - val_loss: 0.0427 - val_accuracy: 

In [57]:
lstm_nadam_optimizer = Nadam(learning_rate=0.001)
lstm_model.compile(optimizer=lstm_nadam_optimizer,
                   loss='binary_crossentropy', metrics=['accuracy'])
lstm_nadam_history = lstm_model.fit(X_train_reshaped_lstm, y_train_lstm,
                                    epochs=30, batch_size=32, validation_data=(X_val_reshaped_lstm, y_val_lstm))


Epoch 1/30
188/188 [==============================] - 3s 7ms/step - loss: 0.3791 - accuracy: 0.4768 - val_loss: 0.0120 - val_accuracy: 0.0000e+00
Epoch 2/30
188/188 [==============================] - 1s 6ms/step - loss: 0.3842 - accuracy: 0.4630 - val_loss: 0.0092 - val_accuracy: 0.0000e+00
Epoch 3/30
188/188 [==============================] - 1s 6ms/step - loss: 0.3847 - accuracy: 0.4743 - val_loss: 0.0145 - val_accuracy: 0.0000e+00
Epoch 4/30
188/188 [==============================] - 1s 6ms/step - loss: 0.3823 - accuracy: 0.4708 - val_loss: 0.0125 - val_accuracy: 0.0000e+00
Epoch 5/30
188/188 [==============================] - 1s 6ms/step - loss: 0.3757 - accuracy: 0.4632 - val_loss: 0.0081 - val_accuracy: 0.0000e+00
Epoch 6/30
188/188 [==============================] - 1s 6ms/step - loss: 0.3780 - accuracy: 0.4410 - val_loss: 0.0085 - val_accuracy: 0.0000e+00
Epoch 7/30
188/188 [==============================] - 1s 6ms/step - loss: 0.3812 - accuracy: 0.4643 - val_loss: 0.0076 - val

In [58]:
lstm_adagrad_optimizer = Adagrad(learning_rate=0.001)
lstm_model.compile(optimizer=lstm_adagrad_optimizer,
                   loss='binary_crossentropy', metrics=['accuracy'])
lstm_adagrad_history = lstm_model.fit(X_train_reshaped_lstm, y_train_lstm,
                                      epochs=30, batch_size=32, validation_data=(X_val_reshaped_lstm, y_val_lstm))


Epoch 1/30


188/188 [==============================] - 3s 7ms/step - loss: 0.3784 - accuracy: 0.4727 - val_loss: 0.0092 - val_accuracy: 0.0000e+00
Epoch 2/30
188/188 [==============================] - 1s 6ms/step - loss: 0.3806 - accuracy: 0.4740 - val_loss: 0.0092 - val_accuracy: 0.0000e+00
Epoch 3/30
188/188 [==============================] - 1s 6ms/step - loss: 0.3807 - accuracy: 0.4713 - val_loss: 0.0092 - val_accuracy: 0.0000e+00
Epoch 4/30
188/188 [==============================] - 1s 6ms/step - loss: 0.3824 - accuracy: 0.4800 - val_loss: 0.0092 - val_accuracy: 0.0000e+00
Epoch 5/30
188/188 [==============================] - 1s 6ms/step - loss: 0.3859 - accuracy: 0.4737 - val_loss: 0.0092 - val_accuracy: 0.0000e+00
Epoch 6/30
188/188 [==============================] - 1s 6ms/step - loss: 0.3806 - accuracy: 0.4748 - val_loss: 0.0092 - val_accuracy: 0.0000e+00
Epoch 7/30
188/188 [==============================] - 1s 6ms/step - loss: 0.3833 - accuracy: 0.4722 - val_loss: 0.0092 - val_accuracy: 

In [59]:
lstm_loss, lstm_accuracy = lstm_model.evaluate(
    X_test_reshaped_lstm, y_test_lstm)
print("Test LSTM Loss: ", lstm_loss)
print("Test LSTM Accuracy: ", lstm_accuracy)


63/63 [==============================] - 0s 2ms/step - loss: 0.0063 - accuracy: 0.0000e+00
Test LSTM Loss:  0.006344666704535484
Test LSTM Accuracy:  0.0


In [60]:
# GRU
X_train_gru, X_temp_gru, y_train_gru, y_temp_gru = train_test_split(
    X_scaled, y, test_size=0.3, random_state=42)

X_val_gru, X_test_gru, y_val_gru, y_test_gru = train_test_split(
    X_temp_gru, y_temp_gru, test_size=0.5, random_state=42)

X_train_reshaped_gru = X_train_gru.reshape(-1, 1, X_train_gru.shape[1])
X_val_reshaped_gru = X_val_gru.reshape(-1, 1, X_val_gru.shape[1])
X_test_reshaped_gru = X_test_gru.reshape(-1, 1, X_test_gru.shape[1])


In [61]:
gru_model = Sequential()
gru_model.add(GRU(units=64, input_shape=(
    X_train_reshaped_gru.shape[1], X_train_reshaped_gru.shape[2]), kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001)))
gru_model.add(Dropout(0.5))


In [62]:
gru_adam_optimizer = Adam(learning_rate=0.001)
gru_model.compile(optimizer=gru_adam_optimizer,
                  loss='binary_crossentropy', metrics=['accuracy'])

gru_adam_history = gru_model.fit(X_train_reshaped_gru, y_train_gru, epochs=30,
                                 batch_size=32, validation_data=(X_val_reshaped_gru, y_val_gru))


Epoch 1/30
219/219 [==============================] - 2s 5ms/step - loss: 0.6540 - accuracy: 0.0309 - val_loss: 0.2103 - val_accuracy: 0.0013
Epoch 2/30
219/219 [==============================] - 1s 4ms/step - loss: 0.4975 - accuracy: 0.1826 - val_loss: 0.1920 - val_accuracy: 0.0000e+00
Epoch 3/30
219/219 [==============================] - 1s 2ms/step - loss: 0.4908 - accuracy: 0.3316 - val_loss: 0.1862 - val_accuracy: 0.0000e+00
Epoch 4/30
219/219 [==============================] - 0s 2ms/step - loss: 0.4848 - accuracy: 0.3861 - val_loss: 0.1823 - val_accuracy: 0.0000e+00
Epoch 5/30
219/219 [==============================] - 0s 2ms/step - loss: 0.4811 - accuracy: 0.3973 - val_loss: 0.1788 - val_accuracy: 0.0133
Epoch 6/30
219/219 [==============================] - 0s 2ms/step - loss: 0.4817 - accuracy: 0.4213 - val_loss: 0.1784 - val_accuracy: 0.0000e+00
Epoch 7/30
219/219 [==============================] - 0s 2ms/step - loss: 0.4822 - accuracy: 0.4289 - val_loss: 0.1766 - val_accurac

In [63]:
gru_nadam_optimizer = Nadam(learning_rate=0.001)
gru_model.compile(optimizer=gru_nadam_optimizer,
                  loss='binary_crossentropy', metrics=['accuracy'])

gru_nadam_history = gru_model.fit(X_train_reshaped_gru, y_train_gru, epochs=30,
                                  batch_size=32, validation_data=(X_val_reshaped_gru, y_val_gru))


Epoch 1/30
219/219 [==============================] - 2s 4ms/step - loss: 0.4770 - accuracy: 0.4349 - val_loss: 0.1699 - val_accuracy: 0.0000e+00
Epoch 2/30
219/219 [==============================] - 1s 2ms/step - loss: 0.4725 - accuracy: 0.4356 - val_loss: 0.1683 - val_accuracy: 0.0000e+00
Epoch 3/30
219/219 [==============================] - 0s 2ms/step - loss: 0.4768 - accuracy: 0.4384 - val_loss: 0.1696 - val_accuracy: 0.0000e+00
Epoch 4/30
219/219 [==============================] - 1s 2ms/step - loss: 0.4758 - accuracy: 0.4467 - val_loss: 0.1690 - val_accuracy: 0.0000e+00
Epoch 5/30
219/219 [==============================] - 1s 3ms/step - loss: 0.4751 - accuracy: 0.4504 - val_loss: 0.1689 - val_accuracy: 0.0000e+00
Epoch 6/30
219/219 [==============================] - 1s 3ms/step - loss: 0.4716 - accuracy: 0.4476 - val_loss: 0.1679 - val_accuracy: 6.6667e-04
Epoch 7/30
219/219 [==============================] - 1s 3ms/step - loss: 0.4752 - accuracy: 0.4501 - val_loss: 0.1693 - val

In [64]:
gru_adagrad_optimizer = Adagrad(learning_rate=0.001)
gru_model.compile(optimizer=gru_adagrad_optimizer,
                  loss='binary_crossentropy', metrics=['accuracy'])

gru_adagrad_history = gru_model.fit(X_train_reshaped_gru, y_train_gru,
                                    epochs=30, batch_size=32, validation_data=(X_val_reshaped_gru, y_val_gru))


Epoch 1/30
219/219 [==============================] - 3s 6ms/step - loss: 0.4761 - accuracy: 0.4384 - val_loss: 0.1678 - val_accuracy: 0.0000e+00
Epoch 2/30
219/219 [==============================] - 0s 2ms/step - loss: 0.4762 - accuracy: 0.4353 - val_loss: 0.1679 - val_accuracy: 0.0000e+00
Epoch 3/30
219/219 [==============================] - 0s 2ms/step - loss: 0.4767 - accuracy: 0.4443 - val_loss: 0.1680 - val_accuracy: 0.0000e+00
Epoch 4/30
219/219 [==============================] - 0s 2ms/step - loss: 0.4729 - accuracy: 0.4484 - val_loss: 0.1681 - val_accuracy: 0.0000e+00
Epoch 5/30
219/219 [==============================] - 0s 2ms/step - loss: 0.4778 - accuracy: 0.4420 - val_loss: 0.1682 - val_accuracy: 0.0000e+00
Epoch 6/30
219/219 [==============================] - 0s 2ms/step - loss: 0.4734 - accuracy: 0.4414 - val_loss: 0.1683 - val_accuracy: 6.6667e-04
Epoch 7/30
219/219 [==============================] - 0s 2ms/step - loss: 0.4764 - accuracy: 0.4556 - val_loss: 0.1683 - val

In [65]:
gru_loss, gru_accuracy = gru_model.evaluate(X_test_reshaped_gru, y_test_gru)
print("Test GRU Loss: ", gru_loss)
print("Test GRU Accuracy: ", gru_accuracy)


47/47 [==============================] - 0s 3ms/step - loss: 0.1789 - accuracy: 0.0000e+00
Test GRU Loss:  0.17891870439052582
Test GRU Accuracy:  0.0


In [66]:
# CNN
X_train_cnn, X_temp_cnn, y_train_cnn, y_temp_cnn = train_test_split(
    X_scaled, y, test_size=0.3, random_state=42)

X_val_cnn, X_test_cnn, y_val_cnn, y_test_cnn = train_test_split(
    X_temp_cnn, y_temp_cnn, test_size=0.5, random_state=42)

print("Shape of X_train:", X_train_cnn.shape)

sequence_length = X_train_cnn.shape[1]
num_features = 1

X_train_reshaped_cnn = X_train_cnn.reshape(-1, sequence_length, num_features)
X_val_reshaped_cnn = X_val_cnn.reshape(-1, sequence_length, num_features)
X_test_reshaped_cnn = X_test_cnn.reshape(-1, sequence_length, num_features)


Shape of X_train: (7000, 17)


In [67]:
cnn_model = Sequential()

cnn_model.add(Conv1D(filters=64, kernel_size=3, activation="relu", input_shape=(
    sequence_length, num_features), kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001)))

cnn_model.add(MaxPooling1D(pool_size=2))

cnn_model.add(Conv1D(filters=32, kernel_size=3, activation="relu"))

cnn_model.add(MaxPooling1D(pool_size=2))

cnn_model.add(Flatten())

cnn_model.add(Dense(128, activation="relu"))

cnn_model.add(Dropout(0.5))


In [68]:
cnn_adam_optimizer = Adam(learning_rate=0.001)
cnn_model.compile(optimizer=cnn_adam_optimizer,
                  loss='binary_crossentropy', metrics=['accuracy'])
cnn_adam_history = cnn_model.fit(X_train_reshaped_cnn, y_train_cnn, epochs=30,
                                 batch_size=32, validation_data=(X_val_reshaped_cnn, y_val_cnn))


Epoch 1/30


219/219 [==============================] - 1s 4ms/step - loss: 0.6214 - accuracy: 0.2436 - val_loss: 0.3707 - val_accuracy: 0.7247
Epoch 2/30
219/219 [==============================] - 1s 4ms/step - loss: 0.5944 - accuracy: 0.8263 - val_loss: 0.3613 - val_accuracy: 0.9113
Epoch 3/30
219/219 [==============================] - 1s 4ms/step - loss: 0.5883 - accuracy: 0.8903 - val_loss: 0.3504 - val_accuracy: 0.9087
Epoch 4/30
219/219 [==============================] - 1s 3ms/step - loss: 0.5801 - accuracy: 0.8997 - val_loss: 0.3424 - val_accuracy: 0.8087
Epoch 5/30
219/219 [==============================] - 1s 2ms/step - loss: 0.5762 - accuracy: 0.9020 - val_loss: 0.3425 - val_accuracy: 0.9540
Epoch 6/30
219/219 [==============================] - 0s 2ms/step - loss: 0.5760 - accuracy: 0.8993 - val_loss: 0.3349 - val_accuracy: 0.9480
Epoch 7/30
219/219 [==============================] - 1s 2ms/step - loss: 0.5700 - accuracy: 0.8961 - val_loss: 0.3247 - val_accuracy: 0.9467
Epoch 8/30
219/21

In [69]:
cnn_nadam_optimizer = Nadam(learning_rate=0.001)
cnn_model.compile(optimizer=cnn_nadam_optimizer,
                  loss='binary_crossentropy', metrics=['accuracy'])
cnn_nadam_history = cnn_model.fit(X_train_reshaped_cnn, y_train_cnn, epochs=30,
                                  batch_size=32, validation_data=(X_val_reshaped_cnn, y_val_cnn))


Epoch 1/30
219/219 [==============================] - 1s 3ms/step - loss: 0.5376 - accuracy: 0.9314 - val_loss: 0.2687 - val_accuracy: 0.9487
Epoch 2/30
219/219 [==============================] - 0s 2ms/step - loss: 0.5378 - accuracy: 0.9403 - val_loss: 0.2696 - val_accuracy: 0.9487
Epoch 3/30
219/219 [==============================] - 0s 2ms/step - loss: 0.5365 - accuracy: 0.9434 - val_loss: 0.2693 - val_accuracy: 0.9473
Epoch 4/30
219/219 [==============================] - 0s 2ms/step - loss: 0.5398 - accuracy: 0.9386 - val_loss: 0.2670 - val_accuracy: 0.8967
Epoch 5/30
219/219 [==============================] - 0s 2ms/step - loss: 0.5336 - accuracy: 0.9267 - val_loss: 0.2632 - val_accuracy: 0.9513
Epoch 6/30
219/219 [==============================] - 0s 2ms/step - loss: 0.5366 - accuracy: 0.9400 - val_loss: 0.2619 - val_accuracy: 0.9513
Epoch 7/30
219/219 [==============================] - 0s 2ms/step - loss: 0.5342 - accuracy: 0.9444 - val_loss: 0.2642 - val_accuracy: 0.9540
Epoch 

In [70]:
cnn_adagrad_optimizer = Adagrad(learning_rate=0.001)
cnn_model.compile(optimizer=cnn_adagrad_optimizer,
                  loss='binary_crossentropy', metrics=['accuracy'])
cnn_adagrad_history = cnn_model.fit(X_train_reshaped_cnn, y_train_cnn, epochs=30,
                                    batch_size=32, validation_data=(X_val_reshaped_cnn, y_val_cnn))


Epoch 1/30
219/219 [==============================] - 1s 2ms/step - loss: 0.5182 - accuracy: 0.9494 - val_loss: 0.2312 - val_accuracy: 0.9547
Epoch 2/30
219/219 [==============================] - 0s 2ms/step - loss: 0.5173 - accuracy: 0.9496 - val_loss: 0.2310 - val_accuracy: 0.9547
Epoch 3/30
219/219 [==============================] - 0s 2ms/step - loss: 0.5168 - accuracy: 0.9496 - val_loss: 0.2309 - val_accuracy: 0.9547
Epoch 4/30
219/219 [==============================] - 0s 2ms/step - loss: 0.5157 - accuracy: 0.9496 - val_loss: 0.2308 - val_accuracy: 0.9547
Epoch 5/30
219/219 [==============================] - 0s 2ms/step - loss: 0.5161 - accuracy: 0.9493 - val_loss: 0.2308 - val_accuracy: 0.9547
Epoch 6/30
219/219 [==============================] - 0s 2ms/step - loss: 0.5178 - accuracy: 0.9496 - val_loss: 0.2307 - val_accuracy: 0.9547
Epoch 7/30
219/219 [==============================] - 0s 2ms/step - loss: 0.5170 - accuracy: 0.9494 - val_loss: 0.2307 - val_accuracy: 0.9547
Epoch 

In [71]:
cnn_loss, cnn_accuracy = cnn_model.evaluate(X_test_reshaped_cnn, y_test_cnn)
print("Test CNN Loss: ", cnn_loss)
print("Test CNN Accuracy: ", cnn_accuracy)


 1/47 [..............................] - ETA: 0s - loss: 2.0721e-04 - accuracy: 1.0000

47/47 [==============================] - 0s 3ms/step - loss: 0.2441 - accuracy: 0.9520
Test CNN Loss:  0.24412576854228973
Test CNN Accuracy:  0.9520000219345093
